In [1]:
import numpy as np
import pandas as pd
import torch
# import torchvision.transforms as transforms

from PIL import Image
from torch.optim import SGD
from tqdm import tqdm

from dataset_class import get_dataloader
# from cnn_class import CnnModel
# from torch.utils.data import DataLoader
# from test_model import CnnModel
from torch.nn.functional import softmax
from torch.utils.tensorboard import SummaryWriter

In [7]:
train_data = pd.read_csv("./data/train1.txt", sep=',')

validation_data = pd.read_csv("./data/validation1.txt", sep=',')

test_data = pd.read_csv("./data/test.txt", sep=',')

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
len(validation_data.id)

1173

In [6]:
def return_pixel_rgb_values(img_name, path = "./data/train_validation/") :
    return np.array(list(Image.open(path + img_name).getdata())).reshape((16, 16, 3))


train_images = np.array([return_pixel_rgb_values(x) for x in train_data.id])/255

validation_images = np.array([return_pixel_rgb_values(x) for x in validation_data.id])/255

test_images = np.array([return_pixel_rgb_values(x, "./data/test/") for x in test_data.id])/255



train_mean_image = np.mean(train_images, axis=(0,1,2))

validation_mean_image = np.mean(validation_images, axis=(0,1,2))

test_mean_image = np.mean(test_images, axis=(0,1,2))


train_std = np.std(train_images, axis=(0,1,2))

validation_std = np.std(validation_images, axis=(0,1,2))

test_std = np.std(test_images, axis=(0,1,2))

In [15]:

train_dataloader = get_dataloader("./data/train_validation/", "./data/train_txt.txt", "train", tuple(train_mean_image), tuple(train_mean_image), batch_size=7)
validation_dataloader = get_dataloader("./data/train_validation/", "./data/validation_txt.txt", "validation", tuple(validation_mean_image), tuple(validation_std), batch_size=7)
# test_dataloader = get_dataloader("./data/test/", "./data/sample_submission.txt", "test", test_mean_image, test_std)


In [8]:
def train_model(model, optimizer, criterion, train_dataloader, epoch, writer) :

    model.train()

    running_loss = 0.0

    for images,labels in tqdm(train_dataloader, "Train"):

        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        running_loss += loss.item() * len(images)

    running_loss /= len(train_dataloader.dataset)

    writer.add_scalar(tag="Loss/Train", scalar_value=running_loss, global_step=epoch)
    writer.flush()

In [9]:
def eval_model(model, validation_dataloader, criterion, epoch, writer, best_loss, second_loss, best_accuracy, second_accuracy, nr, validation = True):
    model.eval()
    running_loss = 0.0
    true_preds = 0

    with torch.no_grad():
        for images,labels in tqdm(validation_dataloader, "Validation"):

            images = images.to(device)
            labels = labels.to(device)

            logits = model(images)

            # Compute the loss
            loss = criterion(logits, labels)

            # Compute accuracy
            scores = softmax(logits, dim=1)

            # Get the prediction labels
            pred_label = torch.argmax(scores, dim=1)

            # Get the acc
            true_preds += (pred_label == labels).sum()

            running_loss += loss.item() * len(images)

        accuracy = true_preds / len(validation_dataloader.dataset)
        print(f"acc is {accuracy}")
        if accuracy > 0.62:
            save_model(model,"./models/", "greater_62")
        if accuracy > 0.605:
            save_model(model,"./models/", "greater_60")
        running_loss = running_loss / len(validation_dataloader.dataset)

        if validation :
            writer.add_scalar(scalar_value=accuracy, tag="Acc/Valid", global_step=epoch)
            writer.add_scalar(scalar_value=running_loss, tag="Loss/Valid", global_step=epoch)
        else :
             writer.add_scalar(scalar_value=accuracy, tag="Acc/Train", global_step=epoch)

        writer.flush()

        if running_loss < best_loss :
             torch.save(model.state_dict(), "./models/best_loss.pth")
             best_loss = running_loss
             print(" am salvat modelul cu best loss")
        elif running_loss < second_loss :
             torch.save(model.state_dict(), "./models/second_loss.pth")
             second_loss = running_loss
             print(" am salvat modelul cu second loss")

        if accuracy > best_accuracy :
             torch.save(model.state_dict(), "./models/best_acc.pth")
             best_accuracy = accuracy
             print(" am salvat modelul cu best acc")
        elif accuracy > second_accuracy :
             torch.save(model.state_dict(), "./models/second_acc.pth")
             second_accuracy = accuracy
             print(" am salvat modelul cu second acc")

In [10]:
def save_model(model, path, model_no):
    torch.save(model.state_dict(), path + f"model{model_no}.pth")
    print("am salvat modelul")


In [11]:
 def run_model(lr, epochs, name_log, nr):
    best_loss = 99999.0
    second_loss = 99999.0
    best_accuracy = 0.0
    second_accuracy = 0.0

    model = CnnModel()

    model = model.to(device)

    optimizer = SGD(model.parameters(), lr = lr)

    criterion = torch.nn.CrossEntropyLoss()

    writer = SummaryWriter(log_dir="./logs/model_" + name_log)

    for epoch in range(epochs) :
        train_model(model, optimizer, criterion, train_dataloader, epoch, writer)
        # train_model(model, optimizer, criterion, validation_dataloader, epoch, writer)
        eval_model(model, validation_dataloader, criterion, epoch, writer, best_loss, second_loss, best_accuracy, second_accuracy, nr, validation=True)
        # eval_model(model, validation_dataloader, criterion, epoch, writer, best_loss, second_loss, best_accuracy, second_accuracy, nr, validation=False)


In [12]:
dataloader_test = get_dataloader("./data/test/", "./data/sample_submission.txt","dani", test_mean_image, test_std)

In [16]:
k = 17

for lr in [0.01, 0.1] :
    epoch = 400
    run_model(lr, epoch, "test_nr" + str(k), k)
    k += 1


Validation: 100%|██████████| 168/168 [00:00<00:00, 286.53it/s]


acc is 0.44075021147727966
 am salvat modelul cu best loss
 am salvat modelul cu best acc


Validation: 100%|██████████| 168/168 [00:00<00:00, 287.54it/s]


acc is 0.47229325771331787
 am salvat modelul cu best loss
 am salvat modelul cu best acc


Validation: 100%|██████████| 168/168 [00:00<00:00, 286.46it/s]


...

acc is 0.6138107776641846
am salvat modelul
 am salvat modelul cu best loss
 am salvat modelul cu best acc


Validation: 100%|██████████| 168/168 [00:00<00:00, 288.27it/s]


acc is 0.5899403095245361
 am salvat modelul cu best loss
 am salvat modelul cu best acc


Train:  21%|██        | 237/1143 [00:01<00:05, 153.40it/s]


KeyboardInterrupt: 

In [17]:
f = open("cnn_62_acc_best_acc.txt", 'w')
f.write("id,label\n")
i = 0

d = {}

model = CnnModel()
model.to(device)
model.load_state_dict(torch.load("./models/modelgreater_62.pth"))

with torch.no_grad():
    for data in dataloader_test:
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        output = model(imgs)
        _, predicteed = torch.max(output.data, 1)

        for label in predicteed :
            f.write(f"{test_data.id[i]},{label}\n")
            i += 1

f.close()

In [ ]:
f = open("cnn_6th_try.txt", 'w')
f.write("id,label\n")
i = 0
net = CnnModel()
net.load_state_dict(torch.load("./model1.pth"))
with torch.no_grad():
    for data in dataloader:
        imgs, labels = data
        output = net(imgs)
        _, predicteed = torch.max(output.data, 1)

        for label in predicteed :
            f.write(f"{test_data.id[i]},{label}\n")
            i+=1

f.close()


In [ ]:
lr = 0.01
epochs = 100

model = CnnModel()
model.to(device)

optimizer = SGD(model.parameters(), lr = lr)

criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs) :

    running_loss = 0.0

    for i,data in enumerate(train_dataloader, 0):

        images,labels = data
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0


In [ ]:
f = open("cnn_third_try.txt", 'w')
f.write("id,label\n")
i = 0

with torch.no_grad():
    for data in dataloader:
        imgs, labels = data
        output = model(imgs)
        _, predicteed = torch.max(output.data, 1)

        for label in predicteed :
            f.write(f"{test_data.id[i]},{label}\n")
            i += 1

f.close()